In [1]:
import stim
import stimcirq

In [8]:
circuit_noisy = stim.Circuit.from_file("./surface_code_bZ_d3_r03_center_3_5/circuit_noisy.stim")

detector_error_model = circuit_noisy.detector_error_model(decompose_errors=True)


In [12]:
def get_detector_val(targets_copy):
    target_val = []
    for target in targets_copy:
        if target.is_relative_detector_id():
            target_val.append(target.val)
    return target_val
def get_logical_observable_val(targets_copy):
    target_val = []
    for target in targets_copy:
        if target.is_logical_observable_id():
            target_val.append(target.val)
    return target_val

def get_flip_index(detector_val, logical_observable_val, detector_number):
    flip_index = []
    for detector in detector_val:
        if detector not in flip_index:
            flip_index.append(detector)
        else:
            flip_index.remove(detector)
    for logical_observable in logical_observable_val:
        logical_observable = logical_observable+detector_number
        if logical_observable not in flip_index:
            flip_index.append(logical_observable)
        else:
            flip_index.remove(logical_observable)
    return flip_index

def flip_detector_even_str_by_flip_index(detector_even_str, flip_index):
    flip_detector_even_str = detector_even_str
    
    for i in flip_index:
        if detector_even_str[i] == '0':
            flip_detector_even_str = flip_detector_even_str[:i]+'1'+flip_detector_even_str[i+1:]
        else:
            flip_detector_even_str = flip_detector_even_str[:i]+'0'+flip_detector_even_str[i+1:]
    
    return flip_detector_even_str
def get_error_model(detector_error_model, detector_number):
    # detector_number = 8
    
    initial_detector_even_str = '0'*(detector_number+1)
    error_model = {}
    error_model[initial_detector_even_str] = 1
    n=0
    for error in detector_error_model:
        print(f"n={n}")
        n = n+1
        if error.type == "error":
            probability = error.args_copy()[0]
            targets_copy = error.targets_copy()
            detector_val = get_detector_val(targets_copy)
            logical_observable_val = get_logical_observable_val(targets_copy)
            flip_index = get_flip_index(detector_val, logical_observable_val, detector_number)
            
            table = {}
            for detector_even_str, detector_even_probability in error_model.items():
                no_flip_detector_even_str = detector_even_str
                no_flip_probability = detector_even_probability * (1-probability)
                
                flip_detector_even_str = flip_detector_even_str_by_flip_index(detector_even_str, flip_index)
                flip_probability = detector_even_probability * probability
                
                table[no_flip_detector_even_str] = table.get(no_flip_detector_even_str, 0) + no_flip_probability
                table[flip_detector_even_str] = table.get(flip_detector_even_str, 0) + flip_probability
            error_model = table
    return error_model

error_model = get_error_model(detector_error_model=detector_error_model, detector_number=detector_error_model.num_detectors)

KeyboardInterrupt: 

In [13]:
len(detector_error_model)

327